In [1]:
# Importamos las librerías necesarias
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from collections import Counter

/Users/felipemendez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Descargamos los datos necesarios de nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/felipemendez/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
def analizar_sentimiento(csv_file, text_column, output_file):
    """
    Realiza un análisis de sentimiento sobre un archivo CSV que contiene una columna de texto y guarda los resultados en un nuevo archivo CSV.
    Para esto se escoge la moda entre 5 modelos de análisis de sentimiento (VADER + 4 transformers)

    Args:
        csv_file (str): Ruta al archivo CSV de entrada que contiene la columna de texto a analizar.
        text_column (str): Nombre de la columna en el CSV que contiene los textos a analizar.
        output_file (str): Ruta al archivo CSV donde se guardarán los resultados del análisis de sentimiento.

    Returns:
        None: Los resultados del análisis de sentimiento se guardan directamente en el archivo `output_file`.
    """
    # Cargamos el archivo CSV
    df = pd.read_csv(csv_file)
    
    # Inicializamos los modelos
    sentiment_pipelines = {
        "xlm_roberta": pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment", tokenizer="cardiffnlp/twitter-xlm-roberta-base-sentiment"),
        "robertuito": pipeline("sentiment-analysis", model="pysentimiento/robertuito-sentiment-analysis", tokenizer="pysentimiento/robertuito-base-uncased"),
        "bert_multilingual": pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", tokenizer="nlptown/bert-base-multilingual-uncased-sentiment"),
        "beto": pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis", tokenizer="finiteautomata/beto-sentiment-analysis")
    }
    
    sia = SentimentIntensityAnalyzer()
    
    # Mapeo de etiquetas para transformers
    def map_sentiment(label):
        mapping = {
            "negative": "negativo", "neutral": "neutral", "positive": "positivo", 
            "1 star": "negativo", "2 stars": "negativo", "3 stars": "neutral", "4 stars": "positivo", "5 stars": "positivo",
            "NEG": "negativo", "NEU": "neutral", "POS": "positivo",
            "LABEL_0": "negativo", "LABEL_1": "neutral", "LABEL_2": "positivo"
        }
        
        return mapping.get(label, "desconocido")
    
    # Función para obtener la moda del sentimiento
    def moda_sentimiento(sentiments):
        conteo = Counter(sentiments)
        return conteo.most_common(1)[0][0]
    
    def obtener_sentimientos(tweet):
        sentimientos = {}
        
        # NLTK (VADER)
        vader_score = sia.polarity_scores(tweet)
        if vader_score['compound'] >= 0.05:
            sentimientos["vader"] = "positivo"
        elif vader_score['compound'] <= -0.05:
            sentimientos["vader"] = "negativo"
        else:
            sentimientos["vader"] = "neutral"
        
        # Transformers
        for model_name, pipeline_model in sentiment_pipelines.items():
            try:
                label = pipeline_model(tweet)[0]['label']
                sentimientos[model_name] = map_sentiment(label)
            except Exception as e:
                print(f"Error con el modelo {model_name}: {e}")
                sentimientos[model_name] = "error"
        
        sentimientos["sentiment_global"] = moda_sentimiento(sentimientos.values())
        return sentimientos
    
    # Aplicamos el análisis de sentimiento
    sentiment_results = df[text_column].astype(str).apply(obtener_sentimientos)
    df_sentiments = pd.DataFrame(list(sentiment_results))
    df = pd.concat([df, df_sentiments], axis=1)
    
    # Guardamos los resultados en un nuevo archivo CSV
    df.to_csv(output_file, index=False)
    print(f"Análisis completado. Resultados guardados en {output_file}")

In [ ]:
# Generamos los resultados
csv_input = "Caso 2 Dataset.csv"
text_column = "texto_tweet"
output_file = "tweets_analizados.csv"
analizar_sentimiento(csv_input, text_column, output_file)